<a href="https://colab.research.google.com/github/swleegit/Hongong_ML/blob/main/05_02%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증세트(validation set)
- 테스트 세트로 모델을 검증하다보면 모델이 테스트 세트에 맞춰지게 된다. 모델을 판단하는 척도의 의미가 사라지게 된다.

- 훈련세트와 테스트세트의 점수 비교를 통해 과대, 과소적합을 판단할 수 있음

- 테스트 세트는 마지막 검증시 딱 한번만 사용하고 훈련세트를 또 나누어서 과대, 과소적합을 판단하면서 진행한다.
<img src = "https://drive.google.com/uc?id=1Kr15JSSY96XSdAIPwB0ddiw1w5v_URM8">


- 보통 20~30%를 테스트세트와 검증세트로 떼어 놓는다. 하지만 문제에 따라 다르다. 훈련데이터가 아주 많다면 단 몇%만 떼어 놓아도 전체 데이터를 대표하는데 문제가 없다.



- 훈련세트에서 모델을 훈련하고 검증세트로 모델을 평가한다. 이런식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다.  
그다음 이 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련데이터에서 모델을 다시 훈련한다.  
그리고 마지막에 테스트 세트에서 최종 점수를 평가한다.

In [44]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")

In [76]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()


In [77]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split\
(data, target, test_size = 0.2, random_state = 42)

In [47]:
#train_input, train_target을 다시 train_test_split으로 훈련세트와 검증세트로 분할한다.
sub_input, val_input, sub_target, val_target = train_test_split\
(train_input, train_target, test_size = 0.2, random_state = 42)


In [48]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [49]:
#훈련세트와 검증세트(validation set)으로 모델을 훈련하고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


0.9971133028626413
0.864423076923077


# 교차검증(cross validation)
- 검증세트를 만드느라 훈련세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다. 그렇다고 검증세트를 너무 조금 떼어 놓으면 검증점수가 들쭉날쭉하고 불안정하다.
- 이럴때 교차검증을 이용하면 안정적인 검증점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

- 교차검증은 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복한다. 그다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 

<img src = "https://drive.google.com/uc?id=1ltKpIoF0SC5lgDeksX3KozFXc2IUKs0a">


- k-fold cross validation  
훈련 세트를 k부분으로 나눠서 교차 검증을 수행하는 것. 보통은 5나 10이다.

- 이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다. 검증세트가 줄얻들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

- 사이킷런에는 cross_validate()라는 교차 검증 함수가 있다.
- 평가할 모델 객체를 첫 번째 매개변수로 전달하고 앞에서 처럼 직접 검증세트를 떼어 내지 않고 훈련세트 전체를 cross_validate()함수에 전달한다.(훈련세트와 테스트 세트를 나누는 과정은 필요)

- scoring 매개변수 : 검증에 사용할 평가 지표를 지정할 수 있다. 기본적으로 분류 모델은 정확도 'accuracy', 회귀 모델은 결정계수를 의미하는 'r2'

- cv 매개변수에 교차 검증 폴드 수나 스플리터 객체를 지정할 수 있다. 

- return_train_score매개변수를 True로 지정하면 훈련세트의 점수도 반환한다(원래는 검증세트 점수만 반환함) 기본값은 False


- 참고 : cross_validate()함수의 전신인 cross_val_score()이 있다. 이 함수는 cross_validate()함수의 결과중에서 test_score만 반환한다.

In [50]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00881243, 0.01040554, 0.00805163, 0.00749922, 0.01015949]), 'score_time': array([0.00087023, 0.00110698, 0.00084901, 0.00078535, 0.00130558]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validatie()라는 함수는 fit_time, score_time, test_score키를 가진 딕셔너리를 반환한다.  
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미  
각 키마다 5개의 숫자(값)이 있다.  
cross_validatie()함수는 기본적으로 5-폴드 교차 검증을 수행하기 때문이다.  
cv 매개변수에서 폴드 수를 바꿀 수도 있다.  
교차 검증의 최종 점수는 test_score키에 담긴 5개의 점수를 평균하여 얻을 수 있다. 이름은 test_score이지만 테스트 세트를 이용한 점수가 아니라 검증세트를 이용한 점수이다! 


In [51]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


한 가지 주의할 점은 cross_validate()는 훈련세트를 섞어 폴드를 나누지 않는다.  
앞서 train_test_split()함수로 전체 데이터를 섞은 후 훈련세트를 준비했기 때문에 따로 섞을 필요가 없다.  
하지만 만약 교차 검증을 할때 훈련세트를 섞으려면 분할기(splitter)를 지정해야한다.  
사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해준다.  
corss_validate()함수는 기본적으로 회귀 모델의 경우 KFOLD분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골구로 나누기 위해 StratifiedKFold를 사용한다.  

- cross_validate()함수에 모델을 첫번째 매개변수로 전달하는데 (이때 모델에는 이미 내가 매개변수를 지정해 놓은 상태) 이 모델이 회귀냐 분류냐에 따라 만약 splitter을 사용한다면 cross_validate()함수의 매개변수 cv가 달라진다는 의미(splitter을 미리 또 지정해놔야 한다)

- splitter을 사용하면 검증세트와 훈련세트를 섞을 수 있다. 

In [52]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [82]:
#만약 훈련세트를 섞은 후 10-폴드 교차검증을 수행하려면 
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
#cross_validate의 k , 섞기, random은 splitter 객체에 
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01402521, 0.01009703, 0.01046801, 0.00935721, 0.00966144,
        0.00978947, 0.00998902, 0.00937033, 0.0097096 , 0.01015329]),
 'score_time': array([0.00101566, 0.00113153, 0.00122285, 0.00098872, 0.00113034,
        0.00118613, 0.00108457, 0.00112057, 0.0010798 , 0.00112629]),
 'test_score': array([0.84807692, 0.89423077, 0.87115385, 0.85576923, 0.86346154,
        0.87884615, 0.87692308, 0.86319846, 0.87668593, 0.87475915])}

# 하이퍼파라미터 튜닝 
모델 파라미터 : 머신러닝 모델이 학습하는 파라미터  
하이퍼파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야하는 것

AutoML : 사람의 개입없이 하이퍼파리미터 튜닝을 자동으로 수행하는 기술  

[중요]
한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾는 방식은 안된다. 예컨대 결정트리에서 max_depth의 최적값은 min_ssamples_split 매개변수의값이 바뀌면 함께 달라진다. 즉 이 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야한다.  

위와 같은 과정은 GridSearchCV클래스를 활용한다.  이는 하이퍼파리미터 탐색과 교차검증을 한번에 수행  
별도로 cross_validate()함수를 호출할 필요가 없다.  
cross_validate()는 교차검증, suffle, random_state, k 지정, 모델 대입, 이때 모델에는 파라미터를 대입한 모델을 대입하는 것  
중요파라미터를 찾기위해서는 for구문을 써서 cross_validate()를 해야함   
이 과정을 이미 만들어놓음 GridSearchCV : 교차검증 + 최적의 하이퍼 파라미터 찾기  
따라서 후보 파라미터를 값으로 대입해야함

In [54]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1) #결정트리 클래스 객체를 생성하자마자 바로 전달 



GridSearchCV의 cv(splitter?) 매개변수 기본값은 5다.  
따라서 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다. 결국 5*5 = 25개의 모델을 훈련한다.(만약 gridsearch 없으면 for과 cross_validate()로 구현)  
많은 모델을 훈련하기 때문에 GridSearchCV클래스의 n_jobs 매개변수에서 병렬 실행에 사용할 cpu코어 수를 지정한다.  
n_jobs = 1 기본값, -1로 지정하면 시스템에 있는 모든 코어를 사용한다




In [55]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

cross_validate의 경우 for을 활용하여 최적의 하이퍼파리머터를 찾았다고 하자. 그러면 전체 훈련세트로 모델을 해당 매개변수로 다시 만들어야한다.  

하지만 gridsearch의 경우 훈련이 끝나면 25개의 모델 중에서 검증점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련한다.   
이 모델은 gs객체의 best_estimator_속성에저장되어 있다.  

grid search 매개변수  
첫번째 매개변수로는 그리드 서치를 수행할 모델 객체를 전달, 두번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달  
scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate()함수와 동일 

In [56]:
dt = gs.best_estimator_ 
#최고의 점수를 받은 매개변수로 gs에서 지정한 학습모델(결정트리)을 전체학습데이터를 활용해서 학습(fit)한것을 
#dt객체에 넘기것 
print(dt.score(train_input,train_target)) #그래서 바로 score을 사용할 수 있다.


0.9615162593804117


In [57]:
#그리더 서치로 찾은 최적의 매개변수는 best_params_속성에 저장되어 있다.
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [58]:
#각 매개변수에서 수행한 교차검증의 평균점수는 cv_results_속성의 'mean_test_score'키에 저장되어 있다.
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [59]:
#배열에서 가장 큰 값의 인덱스를 추출하는 함수
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])
#gs.cv_results_는 딕셔너리 구조, 그중 key가 'params'인것은 value를 리스트로 가지고 있고 best_index로 뽑아낸것

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수를 지정한다.
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
3. 그리더 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련세트가 아니라) 전체 훈련세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.(best_estimator_)

In [60]:
#min_impurity_decrease : 노드를 분할하기 위한 불순도 감소 최소량 지정 
#max_depth ㅍ누기 위한 최소 샘플 수 

#파라미터는 딕셔너리 형태로 지정 
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001) #: 9
,'max_depth' : range(5,20,1), #5,6,7,...19 : 15
'min_samples_split' : range(2,100,10)} #2,12,22...92 : 10
#arange : 실수 ok, range : only 정수 
#경우의 수 9*15*10 = 1350, kfold : 5 : 6750개

In [61]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [62]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [63]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


GridSearchCV클래스를 사용하니 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차검증을 수행해서 최상의 매개변수를 찾을 수 있다.

# 랜덤서치(random search)
- 그리드 서치에서 매개변수의 간격에 대한 근거가 없다. 이보다 더 좁거나 넓은 간격으로 시도할 필요가 있다.

- 매개변수의 값이 수치일 때의 값의 범위나 간격을 미리 정하기 어려운 경우가 많다.

- 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다. 
- 이럴 때 랜덤서치를 사용하면 좋다.

- 랜덤서치에는 매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

- 싸이파이(scipy)는 핵심 과학 라이브러리 중 하나로 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리

In [64]:
from scipy.stats import uniform, randint

- scipy의 stats 서브 패키지에 있는 uniform과 randint클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. = 균등 분포에서 샘플링한다.  
값의 개수가 균등하기보단, 주어진 범위에서 각 구간별 데이터의 개수가 균등하다 = 각 구간별 확률이 균등하게 샘플링하는 것 

- randint는 정수값을 뽑고
- uniform은 실수값을 뽑는다

In [65]:
rgen = randint(0,10) #0과 10범위 내에서 정수를 고르게 샘플링 하도록 하는 객체 만듬 
rgen.rvs() #10개를 균등하게 샘플링 

3

In [66]:
np.unique(rgen.rvs(555), return_counts=True) #고유값, 개수까지 출력
#0~9범위(전체범위)에서 구간별 범위(0~1,1~2...)의 값의 개수가 비슷하게 샘플링 

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([68, 57, 58, 63, 50, 48, 40, 57, 48, 66]))

In [67]:
ugen = uniform(0,100)
np.unique(ugen.rvs(100), return_counts = True)
#0과1사이 실수(전체범위)에서 구간별(정확이 각 구간의 크기는 모르지만)확률이 비슷하도록 값을 샘플링

(array([ 1.0451248 ,  2.50831241,  3.21326628,  3.48224985,  4.08087597,
         4.22898891,  5.85620122,  5.94291316,  8.40547063,  9.57377326,
         9.70222784, 10.30914264, 10.94788174, 12.11316948, 13.05430455,
        14.33249631, 15.24326079, 17.66056432, 18.16822543, 18.97488312,
        20.31435758, 20.33610126, 22.19116936, 24.16595395, 24.55006992,
        25.30586593, 26.0526184 , 26.14326418, 30.71081271, 31.35937187,
        31.5542489 , 32.89294443, 34.95143772, 35.65449827, 37.41213323,
        37.76824878, 38.49607655, 38.62453757, 39.92714545, 42.88925826,
        45.77509346, 45.86810243, 48.58080223, 51.01578038, 51.56833299,
        51.7185875 , 51.87308963, 52.11843455, 52.35446666, 52.70750349,
        54.1187599 , 57.21878207, 57.56522065, 57.67490645, 61.21334821,
        61.79974597, 62.58709301, 63.96387215, 64.1759202 , 67.17705714,
        68.66870983, 68.86874046, 69.89264878, 70.07511381, 70.80543737,
        71.84648492, 72.01551742, 75.9967879 , 76.1

- 랜덤서치에 randint와 uniform 클래스 객체를 넘겨주고 총 몇번을 샘플링해서 최적의 매개변수를 찾으라고 명령. 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋다.


- 탐색할 매개변수의 딕셔너리를 만들자
- min_samples_leaf 매개변수 : 리프 노드가 되기 위한 최소 샘플의 개수, 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다. 

In [70]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [80]:
# from sklearn.model_selection import RandomizedSearchCV
# gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,\
#                         n_iter = 100, n_jobs = -1, random_state = 42)
# gs.fit(train_input, train_target)

from sklearn.model_selection import RandomizedSearchCV

gs= RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

#각 매개변수 100개 sampling 해서 경우의수 100^^4한다는 의미가 아니라
#각 매개변수 균등분포로 정해진 범위안에서 분포해있는데 전체 조합으로 100개 뽑아서 실시한다는것
#gridsearch보다 넓거나 좁은간격으로 넓은 범위의 매개변수에서 적은 시도로 탐색 가능
#컴퓨터 자원이 허락하는한 n_iter이 높으면 좋은 것 
#random_state = 42는 100개를 뽑는 순서에 관련된 것

ValueError: ignored

print(gs.best_params_)  
print(np.max(gs.cv_results['mean_test_score']))  
최적의 모델은 이미 전체 훈련세트 (train_input, train_target)로 훈련되어 best_estimator_속성에 저장되어 있다.  
dt = gs.best_estimator_  
print(dt.score(test_input, test_target))  





랜덤서치는 연속된 매개변수 값을 탐색할 때 유용하다. 탐색할 값을 직접 나열하는 것이 아니라 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달한다. 지정됫 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절 할 수 있다.   